# Setup

In [1]:
import os
from pathlib import Path

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

In [ ]:
from os import listdir
import json
from genericpath import isfile

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Visualization Logic (Dynamic)

In [3]:
def animate_scenario(curr_data: dict, filename: str):
    fig, ax = plt.subplots(1, 1, dpi=300)
    ax.set_xticks([])
    ax.set_yticks([])

    # |Plot roads
    min_x, min_y = (np.inf, np.inf)
    for road_obj in curr_data['roads']:
        road_xy = [(rdict['x'], rdict['y']) for rdict in road_obj['geometry']]
        color = {
            'crosswalk': 'grey',
            'driveway': 'grey',
            'lane': 'darkgrey',
            'road_edge': 'black',
            'road_line': 'yellow',
            'speed_bump': 'grey',
            'stop_sign': 'red'
        }[road_obj['type']]
        rx, ry = zip(*road_xy)
        if min(rx) < min_x: min_x = min(rx)
        if min(ry) < min_y: min_y = min(ry)
        ax.plot(rx, ry, color=color, lw=.2)
    ax.set_title(filename.split('/')[-1])

    # |Get vehicle info
    #2# |Loop through the rest
    for obj_num, veh_obj in enumerate(curr_data['objects']):
        veh_pos = [[posdict['x'],posdict['y']] if posdict['x']!=-10000 else [min_x,min_y] for posdict in veh_obj['position']]
        vx, vy = zip(*veh_pos)
        if obj_num == 0:
            all_vx = np.array([vx])
            all_vy = np.array([vy])
        else:
            all_vx = np.append(all_vx, [vx], axis=0)
            all_vy = np.append(all_vy, [vy], axis=0)

    # Set all vehicles except controlled vehicle to blue
    veh_colors = ['blue']*all_vx.shape[0]
    veh_colors[curr_data['metadata']['sdc_track_index']] = 'red'

    scat = ax.scatter(all_vx[:,0], all_vy[:,0], marker='o', s=veh_obj['width']*.1, lw=1, alpha=1, c=veh_colors)

    # |Plot veh traj
    def animate_func(timestep):        
        curr_offset = np.stack([all_vx[:,timestep],all_vy[:,timestep]]).transpose()
        scat.set_offsets(curr_offset)
    anim = animation.FuncAnimation(fig, animate_func, repeat=True, frames=all_vx.shape[1] - 1, interval=50)

    anim.save(filename=filename, fps=15)
    plt.close()


In [ ]:
# dirpath = "data/processed/construal/"
dirpath = "data/processed/training/"
outputDir = "data/processed/Viz_all/"
scenario_files = [dirpath+fl_name for fl_name in listdir(dirpath)]
for file_num, scFile in enumerate(scenario_files):
    if os.path.isfile(scFile):
        print(f"\rprocesseing file {file_num}", end="", flush=True)
        with open(scFile, 'r') as opn_file:
            data = json.load(opn_file)
        animate_scenario(data, outputDir+scFile.split('/')[-1].replace('.json','.gif'))

processeing file 104

# Visualization Logic (Static)

In [ ]:

def plot_scenario(curr_data: dict, filename: str):
    fig, ax = plt.subplots(1, 1, dpi=300)
    ax.set_xticks([])
    ax.set_yticks([])
    for road_obj in curr_data['roads']:
        road_xy = [(rdict['x'], rdict['y']) for rdict in road_obj['geometry']]
        color = {
            'crosswalk': 'grey',
            'driveway': 'grey',
            'lane': 'darkgrey',
            'road_edge': 'black',
            'road_line': 'yellow',
            'speed_bump': 'grey',
            'stop_sign': 'red'
        }[road_obj['type']]
        rx, ry = zip(*road_xy)
        ax.plot(rx, ry, color=color, lw=.2)
    ax.set_title(filename.split('/')[-1])

    for obj_num, veh_obj in enumerate(curr_data['objects']):
        veh_pos = [[posdict['x'],posdict['y']] for posdict in veh_obj['position'] if posdict['x']!=-10000]
        vx, vy = zip(*veh_pos)
        if obj_num != curr_data['metadata']['sdc_track_index']:
            ax.plot(vx, vy, marker='.', ms=veh_obj['width']*.1, lw=1, alpha=0.3)
        else:
            ax.plot(vx, vy, marker='.', ms=veh_obj['width']*.1, lw=1, alpha=1, color='grey')
    fig.savefig(filename)
    plt.close()


In [ ]:
dirpath = "data/processed/construal/"
outputDir = "data/processed/Viz2/"
scenario_files = [dirpath+fl_name for fl_name in listdir(dirpath)]
for scFile in scenario_files:
    if os.path.isfile(scFile):
        with open(scFile, 'r') as opn_file:
            data = json.load(opn_file)
        plot_scenario(data, outputDir+scFile.split('/')[-1].replace('.json','.png'))

# Extra

### Inspect JSON structure

In [45]:
# Open and read the JSON file
with open('data/processed/construal/tfrecord-00046-of-01000_167.json', 'r') as file:
    path_1 = json.load(file)

# Open and read the JSON file
with open('data/processed/construal/tfrecord-00495-of-01000_351.json', 'r') as file:
    path_2 = json.load(file)

In [46]:
path_1.keys()

dict_keys(['name', 'scenario_id', 'objects', 'roads', 'tl_states', 'metadata'])

In [47]:
path_1['metadata']

{'sdc_track_index': 64,
 'objects_of_interest': [2217, 251],
 'tracks_to_predict': [{'track_index': 11, 'difficulty': 0},
  {'track_index': 64, 'difficulty': 0},
  {'track_index': 29, 'difficulty': 0},
  {'track_index': 33, 'difficulty': 0},
  {'track_index': 19, 'difficulty': 0}]}

In [12]:
path_1['objects'][0].keys()

dict_keys(['position', 'width', 'length', 'height', 'heading', 'velocity', 'valid', 'goalPosition', 'type', 'id', 'mark_as_expert'])

In [48]:
path_1['objects'][0]

{'position': [{'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': 2373.505615234375, 'y': -2837.07958984375, 'z': 65.5370579606807},
  {'x': 2373.4404296875, 'y': -2837.044677734375, 'z': 65.53267476749299},
  {'x': 2373.46875, 'y': -2836.98974609375, 'z': 65.55056252162947},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': -10000.0, 'y': -10000.0, 'z': -10000.0},
  {'x': 2373.395751953125, 'y': -2837.07275390625, 'z': 65.54760728610552},
  {'x': 2373.440673828125, 'y': -2837.04833984375, 'z': 65.55986593833616},
  {'x': 2373.44091796875, 'y'

In [17]:
[obj['mark_as_expert'] for obj in path_1['objects']]

[False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [22]:
path_1['objects'][0]['velocity'][40:]

[{'x': 0.0, 'y': 0.0},
 {'x': 0.0, 'y': 0.0},
 {'x': 0.0, 'y': 0.0},
 {'x': 0.0, 'y': 0.0},
 {'x': 0.0, 'y': 0.0},
 {'x': 0.0, 'y': 0.0},
 {'x': 0.0, 'y': 0.0},
 {'x': 0.0, 'y': 0.0},
 {'x': 0.0, 'y': 0.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 {'x': -10000.0, 'y': -10000.0},
 

In [61]:
with open('data/processed/construal/tfrecord-00046-of-01000_167.json', 'r') as file:
    curr_data = json.load(file)


In [62]:
curr_data['metadata']['sdc_track_index']

64